In [20]:
from cobra.io import load_matlab_model, save_json_model
import numpy as np

# Load the MATLAB model
model = load_matlab_model('./og_model_genome_files/Ax_model_Jan_17_2025.mat')

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, p


In [21]:
model.reactions[0]

Reaction identifier,CYTDK2
Name,Cytidine kinase (GTP)
Memory address,0x7f67fb2cd2a0
Stoichiometry,cytd_c + gtp_c --> cmp_c + gdp_c + h_c Cytidine + GTP C10H12N5O14P3 --> CMP C9H12N3O8P + GDP C10H12N5O11P2 + H+
GPR,222.273.peg.2193
Lower bound,0.0
Upper bound,1000.0


In [16]:
from cobra.io import load_matlab_model, save_json_model
import numpy as np

# Load the MATLAB model
model = load_matlab_model('./og_model_genome_files/Ax_model_Jan_17_2025.mat')

# Function to recursively handle NaN values in nested structures
def replace_nan(obj):
    if isinstance(obj, dict):
        return {k: replace_nan(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [replace_nan(x) for x in obj]
    elif isinstance(obj, (float, np.floating)) and np.isnan(obj):
        return 0.0  # or another appropriate value
    return obj

# Replace NaN values in various model attributes
for reaction in model.reactions:
    # Handle reaction bounds
    if np.isnan(reaction.upper_bound):
        reaction.upper_bound = 1000.0
    if np.isnan(reaction.lower_bound):
        reaction.lower_bound = -1000.0
    
    # Handle other numerical attributes
    for attr in dir(reaction):
        if not attr.startswith('_'):  # Skip private attributes
            try:
                value = getattr(reaction, attr)
            except:
                continue
            if isinstance(value, (float, np.floating)) and np.isnan(value):
                setattr(reaction, attr, 0.0)

for metabolite in model.metabolites:
    # Handle charge
    if hasattr(metabolite, 'charge') and (metabolite.charge is None or (isinstance(metabolite.charge, (float, np.floating)) and np.isnan(metabolite.charge))):
        metabolite.charge = 0
    
    # Handle other numerical attributes
    for attr in dir(metabolite):
        if not attr.startswith('_'):
            try:
                value = getattr(metabolite, attr)
            except:
                continue
            if isinstance(value, (float, np.floating)) and np.isnan(value):
                setattr(metabolite, attr, 0.0)

# Handle model attributes carefully
SKIP_MODEL_ATTRS = {'boundary', 'reactions', 'metabolites', 'genes', 'groups', 'solver'}
for attr in dir(model):
    if (not attr.startswith('_') and 
        attr not in SKIP_MODEL_ATTRS and 
        not callable(getattr(model, attr))):
        try:
            value = getattr(model, attr)
            if is_nan(value):
                setattr(model, attr, 0.0)
        except:
            continue  # Skip any attributes that cause errors


In [1]:
# convert model values
f = open('./inputs/i_ZM4_481.json', 'r')
lines = f.readlines()
f.close()
f = open('./inputs/converted_model.json', 'w')
new_lines = []
for line in lines:
    if 'kegg.gene' in line:
        pre_gene = line.split('zmo')[0]
        gene = line.split('zmo:')[1].split('\"')[0]
        if gene in ZM_gene_to_biocyc:
            new_gene = ZM_gene_to_biocyc[gene]
        else:
            new_gene = gene
        post_gene = '\"'+post_gene.split('\"')[1]
        new_line = pre_gene+new_gene+post_gene
    else:
        new_line = line
    f.write(new_line)
f.close()



# Save as JSON
save_json_model(model, './inputs/converted_model.json')

FileNotFoundError: [Errno 2] No such file or directory: './inputs/i_ZM4_481.json'